# Data analysis of FAMU radioactivity experiment

In [ ]:
import pandas as pd
import glob
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
# Function to select rows based on your pattern
# skip_first=1 means do skip, then select
# skip_first=0 meaan do select, then skip
def select_rows_in_chunks(df, select_size=50, skip_size=50, skip_first=0):
    print('select rows shape ',df.shape)
    selected_rows = []
    for i in range(0, len(df), select_size + skip_size):
        if skip_first == 0:
            selected_rows.extend(range(i, min(i + select_size, len(df))))
            #print('even ',i, min(i + select_size, len(df)))
        else:
            selected_rows.extend(range(i+skip_size, min(i + skip_size + select_size, len(df))))
            #print('odd  ', i+skip_size, min(i + skip_size + select_size, len(df)))
    
    #print('len selected_rows ',len(selected_rows))
    #print(selected_rows)
    
    return df.iloc[selected_rows]
    

In [ ]:
def windowed_diff(s1, s2, n):
    s1ra = s1.rolling(window=n, min_periods=1, center=True).sum()  # min_periods=1 to handle edge cases
    s2ra = s2.rolling(window=n, min_periods=1, center=True).sum()  # min_periods=1 to handle edge cases

    difference = s1ra - s2ra
    #print('s1ra\n',s1ra[400:410])

    errors = np.sqrt(np.abs(s1ra) + np.abs(s2ra))
    
    return difference, errors

    



# Load Data

In [ ]:
# Load datafiles as specified in glob below

#datafiles = glob.glob('PH*.txt')
#datafiles = glob.glob('PH*laser*0001.txt')
#datafiles = glob.glob('PHHvT_Tally4/PH*.txt')
datafiles = glob.glob('Chicago/PH*.txt')

datafiles.sort()
datafiles

# Spectral Data and Differences in Spectra

In [ ]:
# Read data into dataframes
df = {}
tot_spectra = {}      # sum of spectra over all time
even_spectra = {}     # first N minutes, and all the other alternating N minutes thereafter
odd_spectra = {}      # second N minutes
diff = {}             # windowed diff in odd-even spectra
errs = {}             # errors on windowed diff
diff_sigma = {}       # diff in units of sigma
diff_sigma_errs = {}  # error on different

window_size = 1

minutes_onoff = 15
nbins_onoff = minutes_onoff*60//18      # num 18 ns time bins where laser is on, or off
print("nbins on/off = ",nbins_onoff)

for f in datafiles:
    df[f] = pd.read_csv(f, header=None, delimiter=r"\s+")

    nblocks = df[f].index.size // nbins_onoff     # num complete on-off blocks (not cut in middle)
    if (nblocks % 2) == 1:
        print("Have odd number of on-off blocks, dropping last block")
        nblocks = nblocks - 1
    print("nblocks = ",nblocks,", ",f)

    nrows_complete = nblocks*nbins_onoff          # total number of 18ns bins in on or off spectra
    print('nrows_complete ', nrows_complete)
    
    #df[f]
    
    # for each bin, sum the bin over time to get the total spectrum from the run
    tot_spectra[f] = df[f][0:nrows_complete].sum(axis=0)

    # now sum the even and odd in time spectra
    even_spectra[f] = select_rows_in_chunks(df[f][0:nrows_complete],select_size=nbins_onoff,skip_size=nbins_onoff,skip_first=0).sum(axis=0)
    odd_spectra[f] = select_rows_in_chunks(df[f][0:nrows_complete],select_size=nbins_onoff,skip_size=nbins_onoff,skip_first=1).sum(axis=0)

    print('shape even odd ',even_spectra[f].shape,odd_spectra[f].shape)
    
    diff[f], errs[f] = windowed_diff( odd_spectra[f], even_spectra[f], window_size )

    diff_sigma[f] = diff[f]/errs[f]             # diff of spectra 1 and 2 in terms of sigma
    diff_sigma_errs[f] = np.sqrt(1 + 0.5*diff_sigma[f]*diff_sigma[f])     # error on ratio
    

# Time Series

In [ ]:
# Create the time-series for each datafile
counts = {}
time = {}
timestep = 18./3600.   # time in hours

for f in datafiles:
    # get time-series by summing all bins in the spectrum to get the total counts at each time step
    counts[f] = df[f].sum(axis=1)

    # drop last entry since the daq length isn't always 18s
    counts[f] = counts[f].iloc[:-1]

    # get time axis
    tmax = timestep*(len(counts[f][:-1]))
    time[f] = np.linspace(0, tmax, len(counts[f]), endpoint=False)

# Plot Spectra and Differences in Spectra

In [ ]:
docheck = 0

for key,spectra in tot_spectra.items():
    print(key)
    
    #if key != 'PHHvT_020525_sP-sD_B45-PolPer-D1776-B1765_stepmotor.txt':
    #    continue

    plt.figure(figsize=(12, 6))
    plt.xlabel('ADC')
    plt.ylabel('Counts')
    plt.title('Spectrum')
    plt.semilogy(spectra)
    even_spectra[key].plot()
    odd_spectra[key].plot()
    plt.show()

    if docheck == 1:
        print('even_spectra\n',even_spectra[key][400:410])
        print('odd_spectra\n',odd_spectra[key][400:410])
    
    plt.figure(figsize=(12, 6))
    plt.xlabel('ADC')
    plt.ylabel('Delta-Counts')
    plt.title('Laser On - Laser Off Spectrum')
    plt.errorbar(diff[key].index, diff[key], yerr=errs[key], fmt='o', capsize=5, markerfacecolor='green')
    plt.show()

    if docheck == 1:
        print('diff\n',diff[key][400:410])
        print('errs\n',errs[key][400:410])

    plt.figure(figsize=(12, 6))
    plt.xlabel('ADC')
    plt.ylabel('Delta-Counts / Sigma')
    plt.title('Laser On - Laser Off Spectrum, in units of sigma')

    
    if docheck == 1:
        print('diff_sigma\n',diff_sigma[key][400:410])
        print('diff_sigma_errs\n',diff_sigma_errs[key][400:410])

    plt.errorbar(diff[key].index, diff[key]/errs[key], yerr=diff_sigma_errs[key], fmt='o', capsize=5, markerfacecolor='green')
    plt.show()

    # Create Delta/Sigma histogram
    plt.hist(diff_sigma[key], bins=50, edgecolor='black', alpha=0.7)
    plt.xlabel("Delta/Sigma")
    plt.ylabel("Counts")
    plt.title("Histogram of Delta/Sigma")
    plt.show()

    # Plot Time Series
    plt.figure(figsize=(12, 6))
    plt.xlabel('Time [hr]')
    plt.ylabel('Counts/18s')
    plt.title('Counts vs time')
    plt.plot(time[key],counts[key])
    plt.show()


print('END')    

# Fast Fourier Transforms


In [ ]:
# Calculate the FFT of each time series
fft_result = {}
frequencies = {}
pos_freqs = {}
pos_fft = {}

for key,value in counts.items():

    # Perform FFT
    fft_result[key] = np.fft.fft(value)  # Compute FFT
    frequencies[key] = np.fft.fftfreq(len(value),d=timestep)  # Compute frequency bins

    # Only keep the positive half of the spectrum
    pos_freqs[key] = frequencies[key][frequencies[key] >= 0.0]
    pos_fft[key] = np.abs(fft_result[key][frequencies[key] >= 0.0])


 

In [ ]:
# Plot the FFT
for key,value in counts.items():
    print(key)

    plt.figure(figsize=(12, 6))
    plt.xlabel('Freq [/hr]')
    plt.ylabel('FFT Ampl')
    plt.title('FFT')
    plt.plot(pos_freqs[key][1:], pos_fft[key][1:], label="FFT")
    plt.show()



# Save to CSV

In [ ]:
# Save to csv

dfout = {}
row_names = [ 'spectra', 'on spectra', 'off spectra', 'delta','sigma_delta','delta/sigma','sigma_(delta/sigma)' ]

for key,spectra in tot_spectra.items():
    print(key)
    
    dfout[key] = pd.DataFrame(columns=list(range(0,len(spectra))))
    dfout[key].loc[len(dfout[key])] = spectra
    dfout[key].loc[len(dfout[key])] = even_spectra[key]
    dfout[key].loc[len(dfout[key])] = odd_spectra[key]
    dfout[key].loc[len(dfout[key])] = diff[key]
    dfout[key].loc[len(dfout[key])] = errs[key]
    dfout[key].loc[len(dfout[key])] = diff_sigma[key]
    dfout[key].loc[len(dfout[key])] = diff_sigma_errs[key]

    dfout[key].insert(0,'name',row_names)
    
    #print(dfout[key])

    savefname = key[:-4] + '.csv'
    dfout[key].to_csv(savefname)



In [ ]:
#
# Checking that sum is correct... (not off by 1)
#

#fname = 'PHHvT_Tally_laser1_020725_sP-sD_B45-PolPar_D1781_B1761_stepmotor_0001.txt'
#print(df[fname][0:4800].shape)
#print(df[fname][0:4800].sum(axis=0)[0:20])
#print(np.sum(df[fname][0:4800],axis=0)[0:20])

#tot_spectra[fname][0:20]